### 2. Product Funnel Analysis
---
Using a single SQL query - create a new output table which has the following details:

- How many times was each product viewed?
- How many times was each product added to cart?
- How many times was each product added to a cart but not purchased (abandoned)?
- How many times was each product purchased?

In [1]:
DROP TABLE IF EXISTS product_stats;

WITH CTE AS (   
    SELECT distinct visit_id
    FROM events
    WHERE event_type = 1
),
VIEW_CTE AS (   
    SELECT 
        page_name,
        product_category,
        count(page_name) AS viewed
    FROM events e
    LEFT JOIN CTE c ON e.visit_id = c.visit_id 
    JOIN page_hierarchy p ON e.page_id = p.page_id
    WHERE product_category IS NOT NULL AND event_type = 1
    GROUP BY page_name, product_category
),
ADD_TO_CART_CTE AS (   
    SELECT 
        page_name,
        product_category,
        count(page_name) AS added_to_cart
    FROM events e
    LEFT JOIN CTE c ON e.visit_id = c.visit_id 
    JOIN page_hierarchy p ON e.page_id = p.page_id
    WHERE product_category IS NOT NULL AND event_type = 2
    GROUP BY page_name, product_category
),
PURCHASE_CTE AS(
    SELECT visit_id
    FROM events
    WHERE event_type = 3
),
PURCHASE_CTE_2 AS(
    SELECT
        page_name,
        product_category,
        COUNT(*) as purchased
    FROM events e
    JOIN PURCHASE_CTE pc ON pc.visit_id = e.visit_id
    JOIN page_hierarchy p ON p.page_id = e.page_id
    WHERE event_type = 2
    GROUP BY page_name, product_category
)
SELECT
    v.page_name,
    v.product_category,
    viewed,
    added_to_cart,
    purchased,
    (added_to_cart - purchased) as abandoned
INTO product_stats
FROM VIEW_CTE v
LEFT JOIN ADD_TO_CART_CTE A ON v.page_name = a.page_name
LEFT JOIN PURCHASE_CTE_2 P ON v.page_name = p.page_name
ORDER BY v.page_name


(9 rows affected)

Total execution time: 00:00:00.082

Additionally, create another table which further aggregates the data for the above points but this time for each product category instead of individual products.

In [2]:
DROP TABLE IF EXISTS product_category_stats
SELECT
    product_category,
    SUM(viewed) AS viewed,
    SUM(added_to_cart) AS added_to_cart,
    SUM(purchased) AS purchased,
    SUM(abandoned) AS abandoned
INTO product_category_stats
FROM product_stats
GROUP BY product_category

(3 rows affected)

Total execution time: 00:00:00.009

Use your 2 new output tables - answer the following questions:

    1. Which product had the most views, cart adds and purchases?

In [3]:
WITH tbl AS(
    SELECT *,
        RANK() OVER(ORDER BY viewed DESC) as rank_viewed,
        RANK() OVER(ORDER BY added_to_cart DESC) as rank_added_to_cart,
        RANK() OVER(ORDER BY purchased DESC) as rank_purchased
    FROM product_stats
)
SELECT *
FROM tbl
WHERE rank_viewed = 1 OR rank_added_to_cart = 1 OR rank_purchased = 1

(2 rows affected)

Total execution time: 00:00:00.022

page_name,product_category,viewed,added_to_cart,purchased,abandoned,rank_viewed,rank_added_to_cart,rank_purchased
Lobster,Shellfish,1547,968,754,214,6,1,1
Oyster,Shellfish,1568,943,726,217,1,4,2


    2. Which product was most likely to be abandoned?

In [4]:
SELECT TOP 1
    page_name,
    abandoned
FROM product_stats
ORDER BY abandoned DESC


(1 row affected)

Total execution time: 00:00:00.020

page_name,abandoned
Russian Caviar,249


    3.  Which product had the highest view to purchase percentage?

In [5]:
SELECT TOP 1
    page_name,
    viewed,
    purchased,
    (100*CAST(purchased as float)/viewed) AS view_to_purchase_percentage
FROM product_stats
ORDER BY view_to_purchase_percentage DESC

(1 row affected)

Total execution time: 00:00:00.019

page_name,viewed,purchased,view_to_purchase_percentage
Lobster,1547,754,48.739495798319325


    4. What is the average conversion rate from view to cart add?

In [6]:
SELECT
    ROUND(AVG(100*CAST(added_to_cart as float)/viewed),2) AS avg_converstion_rate_from_view_to_cart_add
FROM product_stats

(1 row affected)

Total execution time: 00:00:00.014

avg_converstion_rate_from_view_to_cart_add
60.95


    5. What is the average conversion rate from cart add to purchase?

In [7]:
SELECT
    ROUND(AVG(100*CAST(purchased as float)/added_to_cart),2) AS avg_converstion_rate_from_cart_add_to_purchase
FROM product_stats

(1 row affected)

Total execution time: 00:00:00.014

avg_converstion_rate_from_cart_add_to_purchase
75.93
